In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import datetime
import math
from scipy.optimize import root
import glob
import os

In [8]:
def c0(x, dt, k):
    return (-2 * x + (dt / k)) / (2 * (1 - x) + (dt / k))
            
def c1(x, dt, k):
    return (2 * x + (dt / k)) / (2 * (1 - x) + (dt / k))

def c2(x, dt, k):
    return (2 * (1 - x) - (dt / k)) / (2 * (1 - x) + (dt / k))

def combine(x, dt, k):
    c_0, c_1, c_2 = c0(x, dt, k), c1(x, dt, k), c2(x, dt, k)
    if math.isclose(c_0 + c_1 + c_2, 1):
        return c_0, c_1, c_2
    else:
        raise warnings.warn(f"Issue with coding: {c_0}, {c_1}, {c_2} = {c_0+ c_1 +c_2}")
    
def prev_date(date, days=1):    
    "function to get prev timestep"
    subtracted_date = pd.to_datetime(date) - datetime.timedelta(days=days)
    subtracted_date = subtracted_date.strftime("%Y-%m-%d")
    return subtracted_date

In [88]:
def obj(params, data, return_df=False):
    x, k = params
    df, dt = data[0], data[1]
    c_0, c_1, c_2 = combine(x, dt, k)
    
    for i, date in enumerate(df.index):
        if i == 0:
            pass
        else:
            day_before = prev_date(date, days=dt)
            if (day_before in df_up.index):
                i1 = df.loc[day_before,"I"]  # inflow timestep before
                o1 = df.loc[day_before, "O"] # outflow timestep before
                i2 = df.loc[date, "I"]
                o2 = c_0 * i2 + c_1 * i1 + c_2 * o1
                df.loc[date, "Qm"] = o2
                df.loc[date, "RMSE"] = (df.loc[date, "Qm"] - df.loc[date, "O"])**2
            else:
                break
    RMSE_sum = np.sqrt(df["RMSE"].sum())/ (len(df["RMSE"].dropna()))
    if return_df:
        return df
    return RMSE_sum,RMSE_sum
            
    

In [89]:
path = os.getcwd()
home_path = os.path.dirname(path)
data_folder = f'{home_path}\\Data'
files = glob.glob(f"{data_folder}\\*.parquet")
files

['C:\\Users\\anne-\\OneDrive - Delft University of Technology\\Documenten\\Environmental Engineering MSc\\ENVM1502-Catchment-model\\Data\\combined_data.parquet']

In [90]:
df_up = pd.read_parquet(files, columns=['Q']).dropna().rename(columns={'Q':'I'})
df_dw = pd.read_parquet(files, columns=['Q']).dropna().rename(columns={'Q':'O'})

In [91]:
dt = 1
df = pd.concat([df_up, df_dw],axis=1)
data = [df, dt]
params = [0.2, 1]

In [87]:
output = root(obj,(0.23, .63), args=(data))
output

 message: The solution converged.
 success: True
  status: 1
     fun: [ 4.229e-15  4.229e-15]
       x: [-1.913e+00 -5.502e-13]
    nfev: 27
    fjac: [[-7.071e-01 -7.071e-01]
           [ 7.071e-01 -7.071e-01]]
       r: [-2.334e-06  1.088e-02 -2.340e-16]
     qtf: [-8.565e-12  1.842e-25]

In [ ]:
fig, ax = plt.subplots(1)
df_out = obj(output.x, data, return_df=True)[["Qm", "O", "I"]]
df_out.plot(ax=ax)
ax.set_title("Modeled vs actual discharge peak")
ax.legend(["Modelled","Measured","Previous station"])

In [80]:
df_up.loc[]

array([False, False, False, ..., False, False, False])

In [82]:

    print ('date exist')

date exist
